In [ ]:
from dotenv import load_dotenv
import finnhub
import logging
import os
import pandas as pd
import time

try:
    os.remove("stocks.csv")
except:
    pass
cur_date = ""

load_dotenv()
API_KEY = os.getenv("FINNHUB_API")
API_KEY2 = os.getenv("FINNHUB2_API")
API_KEY3 = os.getenv("FINNHUB3_API")

finnhub_client = finnhub.Client(api_key=f"{API_KEY}")
alt_finnhub_client = finnhub.Client(api_key=f"{API_KEY2}")
alt2_finnhub_client = finnhub.Client(api_key=f"{API_KEY3}")
clients = [finnhub_client, alt_finnhub_client, alt2_finnhub_client]

SYMBOLS = finnhub_client.stock_symbols("US")
SYMBOLS = sorted(
    [item for item in SYMBOLS if item["type"] == "Common Stock"],
    key=lambda x: x["symbol"],
)
STOCKS = {}

period = 0
max_retries = 3
retry_delay = 1
for count, symbol in enumerate(SYMBOLS):
    retries = 0
    while retries < max_retries:
        try:
            client = clients[count % len(clients)]
            recommendations = client.recommendation_trends(symbol["symbol"])

            if count == 0:
                print(f"LAST UPDATED: {recommendations[period]['period']}")
                cur_date = recommendations[period]["period"]
            if count % 2000 == 0:
                print(f"{count} out of {len(SYMBOLS)} complete")

            STOCKS[symbol["symbol"]] = 0
            STOCKS[symbol["symbol"]] += 2 * recommendations[period]["strongBuy"]
            STOCKS[symbol["symbol"]] += 1 * recommendations[period]["buy"]
            STOCKS[symbol["symbol"]] += -1 * recommendations[period]["sell"]
            STOCKS[symbol["symbol"]] += -2 * recommendations[period]["strongSell"]

            break
        except Exception as error:
            if str(error) == "list index out of range":
                break
            logging.error(f"Error processing {symbol['symbol']}: {error}")
            retries += 1
            time.sleep(retry_delay)
    time.sleep(1.075)

sorted_stocks = dict(sorted(STOCKS.items(), key=lambda item: item[1], reverse=True))
for symbol, score in list(sorted_stocks.items())[:10]:
    print(f"{symbol}: {score}")
print(len(sorted_stocks))

sorted_stocks = list(sorted_stocks.items())
df = pd.DataFrame(sorted_stocks, columns=["symbol", "score"])
df["cur_date"] = cur_date
df.to_csv("stocks.csv", index=False)
df.to_csv("../../../trevorAppsWebsites/daily-stockz/src/data/stocks.csv", index=False)